# 베드락을 이용하여 리뷰, 상담내역 요약 및 분석 요약하기

> *이 노트북은 SageMaker Studio*의 `JupyterLab` 에서 테스트하였습니다. 

## 소개

이 노트북에서는 리테일 고객이 크기가 큰 문서를 요약하는 방법을 보여드리겠습니다.대용량 문서로 작업할 때 입력 텍스트가 모델 컨텍스트 길이에 맞지 않거나, 모델이 대용량 문서를 인식하지 못하거나, 메모리 부족 오류 등으로 인해 몇 가지 문제에 직면할 수 있습니다. 이러한 문제를 해결하기 위해 청킹 및 연쇄 프롬프트 개념을 기반으로 하는 아키텍처를 보여드리겠습니다. 이 아키텍처는 언어 모델로 구동되는 애플리케이션을 개발하는 데 널리 사용되는 프레임워크인 LangChain을 활용합니다.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html)은 언어 모델로 구동되는 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크의 핵심 측면을 통해 다양한 구성 요소를 연결하여 고급 사용 사례를 만들어 대규모 언어 모델을 보강할 수 있습니다.


#### 컨텍스트

이 노트북에서는 LangChain 프레임워크 내에서 Amazon Bedrock과 통합하여 사용는 방법과 PromptTemplate의 도움으로 텍스트를 생성하는 데 어떻게 사용될 수 있는지 살펴보겠습니다.


#### 사용사례

이 접근 방식은 통화 녹취록, 회의 녹취록, 책, 기사, 블로그 게시물 및 상품/서비스 관련 콘텐츠를 요약하는 데 사용할 수 있습니다.


#### 구현 방법

이 사용 사례를 보여주기 위해 이 노트북에서는 고객의 이전 제품 설명을 기반으로 신규 제품 설명을 생성하는 방법을 보여드리며, Boto3 클라이언트와 함께 Amazon Bedrock API를 사용하는 Anthropic Claude 모델을 사용하겠습니다.


## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) 베드락에 연결해야 합니다.

우선 사전에 설치가 필요한 패키지들을 설치하세요. 그 이후에 셋업에 필요한 라이브러리들을 설치합니다. 

#### 앞 부분은 이전 실습 과정에서 했던 내용과 동일합니다

In [ ]:
!pip install -r dependencies/requirements.txt

In [ ]:
import json
import os
import sys
import boto3
import botocore
from botocore.config import Config
from langchain_community.chat_models import BedrockChat
from langchain.chains.summarize import load_summarize_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# this is setting the maximum number of times boto3 will attempt our call to bedrock
my_region = "us-east-1" # change this value to point to a different region
my_config = Config(
    region_name = my_region,
    signature_version = 'v4',
    retries = {
        'max_attempts': 3,
        'mode': 'standard'
    }
)

# this creates our client we will use to access Bedrock
bedrock_rt = boto3.client("bedrock-runtime", config = my_config)
bedrock = boto3.client("bedrock")

## 베드락 LLM 모델 호출하기

LLM에서 Bedrock 클래스의 인스턴스를 생성하는 것으로 시작하겠습니다. 여기에는 Amazon Bedrock에서 사용할 수 있는 모델의 ARN인 model_id가 필요합니다.

선택적으로 이전에 생성한 boto3 클라이언트를 전달할 수 있으며, `temperature`, `top_p`, `max_tokens` 또는 `stop_sequences`와 같은 매개 변수를 보유할 수 있는 일부 `model_kwargs`도 전달할 수 있습니다(매개 변수에 대한 자세한 내용은 Amazon Bedrock 콘솔에서 살펴볼 수 있습니다).

Amazon Bedrock에서 사용 가능한 텍스트 생성 모델 ID에 대한 [설명서](https://docs.aws.amazon.com/ko_kr/bedrock/latest/userguide/model-ids-arns.html)를 확인하세요.

모델마다 지원하는 `model_kwargs`가 다르다는 점에 유의하세요.

In [ ]:
sonnet_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
# to switch to claude v3 Haiku you can use this id
haiku_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["Human"],
}

# this defines a sonnet object
llm = BedrockChat(
    client=bedrock_rt,
    model_id=sonnet_model_id,
    model_kwargs=model_kwargs,
)

# for any of our chains we can change out the model by simply substituting the sonnet model object with this haiku object.
haiku_model = BedrockChat(
    client=bedrock_rt,
    model_id=haiku_model_id,
    model_kwargs=model_kwargs,
)

## 리뷰 텍스트 요약하기


In [ ]:
%%time
# Invoke Example, request is a variable we must fill in when we invoke our chain
messages = [
    ("user", "{request}"),
]

# pass our list of messages to our ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(messages)

# use | to chain together multiple components
chain = prompt | llm 

# request 
request = """

Human: 다음 텍스트를 3줄로 요약하세요.
<text>
1. 핏이 좋고 편안합니다.하지만 세탁 후 색이 번졌어요.그리고 권장대로 세탁했어요.
이제 사방에 커다란 분홍색 얼룩이 생겼어요.
2. 너무 편안하고 귀엽고 스타일리시합니다.마음에 들어요!
3. 이 스웨터가 마음에 들어요.소재는 훌륭하지만 조금 짧았습니다.
4. 분명히 위험하다는 건 알았지만 다른 사람들에게서 봤을 때 위글 공간이 조금 더 있을지도 모른다고 생각했지만 XL이 생겼고 예상대로 더 오버사이즈였으면 좋겠어요.
그러니 덩치가 큰 제 딸들에게는 생각처럼 오버사이즈가 아니라는 걸 명심하세요!아직 엄청 귀엽고 부드러워요. 하지만 이거 자연 건조할게요!!!
5. 마음에 들어요, 멋진 오버사이즈 핏, 귀여운 색상!!
</text>

Assistant:"""

# Chain Invoke
response = chain.invoke({"request": request})
response.content

## 리뷰 텍스트 요약하기

PDF 문서를 로드한 후에, 긴 문서를 청킹(chunking)합니다. 

In [ ]:
pdf_path = "data/2022-Shareholder-Letter-ko.pdf"

In [ ]:
loader = PyPDFLoader(file_path=pdf_path)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " "], chunk_size=4000, chunk_overlap=100 
)

docs = text_splitter.split_documents(documents=documents)

In [ ]:
print(docs)

## Summarizing chunks and combining them

다른 문서에서 토큰의 수가 일정하다고 가정하면 문제가 없을 것입니다. LangChain의 load_summarize_chain을 사용하여 텍스트를 요약해 보겠습니다. load_summarize_chain은 stuff, map_reduce, refine의 세 가지 요약 방법을 제공합니다.

* stuff는 모든 청크를 하나의 프롬프트에 넣습니다. 따라서 토큰의 최대 한도에 도달하게 됩니다.
* map_reduce는 각 청크를 요약하고, 요약을 결합한 다음, 결합된 요약을 요약합니다. 결합된 요약이 너무 크면 오류가 발생할 수 있습니다.
* refine은 첫 번째 청크를 요약한 다음 첫 번째 요약으로 두 번째 청크를 요약합니다. 모든 청크가 요약될 때까지 동일한 프로세스가 반복됩니다.

map_reduce와 refine은 LLM을 여러 번 호출하므로 최종 요약을 얻는 데 시간이 걸립니다. 여기에서는 stuff는를 사용해 보겠습니다.

'map_reduce' 체인은 큰 문서를 관리하기 쉬운 작은 덩어리로 분할하여 문서 처리를 처리하도록 설계되었습니다. 이 체인은 각 조각에 초기 프롬프트를 사용하여 문서의 특정 섹션을 기반으로 요약 또는 답변을 생성합니다. 또한 MapReduceDocumentsChain은 생성된 출력을 가져와 다른 프롬프트를 사용하여 결합하여 전체 문서에 대한 포괄적이고 일관된 요약 또는 답변을 생성합니다. load_summarize_chain 함수를 사용하여 'map_reduce' 체인을 설정하고 출력 요약을 얻습니다.

In [ ]:
from langchain.prompts import PromptTemplate
import textwrap

chain = load_summarize_chain(llm, chain_type="stuff",verbose=True)
output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)



## 요약 결과 분석하기 


In [ ]:
print(output_summary)

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

prompt = f""" 아래 text의 내용이 (1) 긍정 (2) 부정 (3) 중립 중에 어디에 속하는지 알려주세요. 다른 설명은 하지 말고 바로 긍정, 부정, 중립 이 단어로만 답하세요.:
"{output_summary}"
"""

response = llm.invoke(prompt).content #프롬프트에 응답을 반환
print(response)

## Streamlit 어플리케이션 수행하기

어플리케이션 수행을 위한 스크립트 준비하기

In [ ]:
%%writefile ../summarization_lib.py

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import BedrockChat
from langchain.chains.summarize import load_summarize_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader


def get_llm():
    
    model_kwargs =  { #Anthropic 모델
        "max_tokens": 4000, 
        "temperature": 0
        }
    
    llm = BedrockChat(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0", #파운데이션 모델 설정하기
        model_kwargs=model_kwargs) #Claude의 속성을 구성합니다.
    
    return llm


pdf_path = "PromptEngineering/data/2022-Shareholder-Letter-ko.pdf"

#pdf_path = "2022-Shareholder-Letter.pdf"

def get_docs():
    
    loader = PyPDFLoader(file_path=pdf_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", " "], chunk_size=4000, chunk_overlap=100 
    )
    docs = text_splitter.split_documents(documents=documents)
    
    return docs



def get_summary():
          
    llm = get_llm()
    docs = get_docs()
    
    chain = load_summarize_chain(llm, chain_type="stuff",verbose=True)

    return chain.invoke(docs, return_only_outputs=True)


def get_analysis():

    llm = get_llm()
    summary = get_summary()
    
    input_content= f""" 아래 text의 내용이 (1) 긍정 (2) 부정 (3) 중립 중에 어디에 속하는지 알려주세요. 다른 설명은 하지 말고 바로 긍정, 부정, 중립 이 단어로만 답하세요.:
        "{summary}"
    """
    response = llm.invoke(input_content).content
    return response
    

In [ ]:
%%writefile ../summarization_app.py
import streamlit as st
import summarization_lib as glib

st.set_page_config(layout="wide", page_title="문서 요약")
st.title("문서 요약 및 감정 분석")

#return_intermediate_steps = st.checkbox("중간 단계 반환", value=True)
summarize_button = st.button("요약 및 분석", type="primary")

if summarize_button:
    #st.subheader("요약 및 분석")

    with st.spinner("Running..."):
        response_content = glib.get_summary()
        analysis_content = glib.get_analysis()

        st.write(response_content["output_text"])
        st.write(analysis_content)
    